In [1]:
from muselsl import *
import numpy as np
import matplotlib.pyplot as plt
from pylsl import StreamInlet, resolve_byprop
import json
import warnings

import xml.dom.minidom

In [10]:
# -*- coding: utf-8 -*-
"""
Estimate Relaxation from Band Powers
This example shows how to buffer, epoch, and transform EEG data from a single
electrode into values for each of the classic frequencies (e.g. alpha, beta, theta)
Furthermore, it shows how ratios of the band powers can be used to estimate
mental state for neurofeedback.
The neurofeedback protocols described here are inspired by
*Neurofeedback: A Comprehensive Review on System Design, Methodology and Clinical Applications* by Marzbani et. al
Adapted from https://github.com/NeuroTechX/bci-workshop
"""

import numpy as np  # Module that simplifies computations on matrices
import matplotlib.pyplot as plt  # Module used for plotting
from pylsl import StreamInlet, resolve_byprop  # Module to receive EEG data
import utils  # Our own utility functions
from process_data import ChannelDataProcessor, Metrics


# Handy little enum to make code more readable


class Band:
    Delta = 0
    Theta = 1
    Alpha = 2
    Beta = 3


# We are working with 4 channels (Billy) [0], [1], [2], [3] as 4 index_channel values
NUM_CHANNELS = 5

# Length of the EEG data buffer (in seconds)
# This buffer will hold last n seconds of data and be used for calculations
BUFFER_LENGTH = 5

# Length of the epochs used to compute the FFT (in seconds)
EPOCH_LENGTH = 1

# Amount of overlap between two consecutive epochs (in seconds)
OVERLAP_LENGTH = 0.8

# Amount to 'shift' the start of each next consecutive epoch
SHIFT_LENGTH = EPOCH_LENGTH - OVERLAP_LENGTH


def _acquire_eeg_data(_inlet):
    """ Pull EEG data from inlet and return. 

    :return: tuple: _eeg_data, _timestamp
    """
    _eeg_data, _timestamp = _inlet.pull_chunk(
        timeout=1, max_samples=int(SHIFT_LENGTH * fs))
    return _eeg_data, _timestamp

def _acquire_eeg_data_mock():
    """ Get eeg_data and timestamp from json for testing purposes. 

    :return: tuple: _eeg_data, _timestamp
    """
    json_file = open("to_send.json")
    k = json.loads(json_file.read())
    _eeg_data = k[0]["eeg_data"]
    _timestamp = k[0]["timestamp"]
    return _eeg_data, _timestamp

if __name__ == "__main__":

    # """ 1. CONNECT TO EEG STREAM """
    # 
    # Search for active LSL streams
    print('Looking for an EEG stream...')
    streams = resolve_byprop('type', 'EEG', timeout=2)
    if len(streams) == 0:
        raise RuntimeError('Can\'t find EEG stream.')
    
    # Set active EEG stream to inlet and apply time correction
    print("Start acquiring data")
    inlet = StreamInlet(streams[0], max_chunklen=12)
    eeg_time_correction = inlet.time_correction()
    
    # Get the stream info and description
    info = inlet.info()
    description = info.desc()
    
#     dom = xml.dom.minidom.parseString(description.value())
#     print(info.channel_count)
#     print(description.value())
#     print(dom.toprettyxml())
# # description.value
    
    # # Get the sampling frequency
    # # This is an important value that represents how many EEG data points are
    # # collected in a second. This influences our frequency band calculation.
    # # for the Muse 2016, this should always be 256
    
    def get_fs(_info):
        return int(_info.nominal_srate())
    
    def get_fs_mock():
        warnings.warn("Using mock fs value: 256. Make sure that you are using get_fs outside of testing. ")
        return 256  # For testing purposes
    
    fs = get_fs(info)
#     fs = get_fs_mock() 

    """ 3. GET DATA """

    # The try/except structure allows to quit the while loop by aborting the
    # script with <Ctrl-C>
    print('Press Ctrl-C in the console to break the while loop.')
    deltas = []
    thetas = []
    alphas = []
    betas = []
    try:
        # The following loop acquires data, computes band powers, and calculates neurofeedback metrics based on those band powers
        while True:
            #
            """ 3.1 ACQUIRE DATA """
            
            # Obtain EEG data from the LSL stream
            eeg_data, timestamp = _acquire_eeg_data(inlet) 
#             eeg_data, timestamp = _acquire_eeg_data_mock()
            
            c = ChannelDataProcessor(buffer_length=BUFFER_LENGTH, 
                                     epoch_length=EPOCH_LENGTH, 
                                     overlap_length=OVERLAP_LENGTH, 
                                     shift_length=SHIFT_LENGTH, fs=fs, band_cls=Band)
            
            c.feed_new_data(eeg_data=eeg_data)          # Feed new data generated in the epoch
            metrics = np.zeros(NUM_CHANNELS)                       
            delta_sample = []
            theta_sample = []
            alpha_sample = []
            beta_sample = []
            for i in range(NUM_CHANNELS):               # Iterate through all separate channels

                
                # Record channel smooth band power
                csbp = c.get_channel_smooth_band_powers(i)
                delta_sample.append(csbp[0])
                theta_sample.append(csbp[1])
                alpha_sample.append(csbp[2])
                beta_sample.append(csbp[3])
                
                # Run calculations on csbp to obtain desired metrics
                beta_metric = Metrics.beta_protocol(csbp, Band)
                #print("Alpha metric: {}".format(alpha_metric))
                
                metrics[i] = beta_metric

            #print("alpha metric for 4 sensors are separately: {}".format(metrics))
            print(" ")
            deltas.append(delta_sample)
            thetas.append(theta_sample)
            alphas.append(alpha_sample)
            betas.append(beta_sample)
            
    except KeyboardInterrupt:
        print('Closing!')


Looking for an EEG stream...
Start acquiring data
Press Ctrl-C in the console to break the while loop.
Beta Concentration:  -0.8198875062685185
Beta Concentration:  0.40239030343812604
Beta Concentration:  0.3533434665369567
Beta Concentration:  -0.803726814270128
Beta Concentration:  2.8630071509960184
 
Beta Concentration:  -1.1577084016729502
Beta Concentration:  3.0349351507056266
Beta Concentration:  6.746269040862719
Beta Concentration:  -0.6685282437627267
Beta Concentration:  0.7595000428768672
 
Beta Concentration:  -0.11976901997060375
Beta Concentration:  0.8097883925745597
Beta Concentration:  0.729865886403896
Beta Concentration:  -0.009366099574849043
Beta Concentration:  2.81577569811154
 
Beta Concentration:  -4.721807246668743
Beta Concentration:  0.6339267356765746
Beta Concentration:  0.49640558221274267
Beta Concentration:  -4.140878359142744
Beta Concentration:  8.179887354135165
 
Beta Concentration:  1.725567660813634
Beta Concentration:  0.5825272859656794
Beta 

Beta Concentration:  3.8446850168322038
Beta Concentration:  2.2649098729371575
Beta Concentration:  0.7471654415243864
Beta Concentration:  2.582803166707071
Beta Concentration:  1.7256742289773543
 
Beta Concentration:  -2.0244526789359036
Beta Concentration:  0.6780984271718432
Beta Concentration:  0.41990314338898166
Beta Concentration:  -2.5637955110870236
Beta Concentration:  0.9617454327985485
 
Beta Concentration:  0.20164193252470528
Beta Concentration:  0.5498175955840776
Beta Concentration:  0.5556550759855401
Beta Concentration:  0.16448210218844894
Beta Concentration:  0.43443047649214295
 
Beta Concentration:  -0.7324681740969476
Beta Concentration:  1.137138786395201
Beta Concentration:  0.38177847798438874
Beta Concentration:  -0.675229649443261
Beta Concentration:  3.3932891990784513
 
Beta Concentration:  -0.101572893663242
Beta Concentration:  0.39855764129707066
Beta Concentration:  0.4115612944260834
Beta Concentration:  -0.09291674439910733
Beta Concentration:  2.

In [14]:
thetas

[[0.027080094013804474,
  0.04307969962285625,
  0.04525204138904593,
  0.02820186549381042,
  -0.004988279819932072],
 [0.01289965989733113,
  0.0064893059633092534,
  0.002730737590208447,
  0.01620731016278457,
  -0.01993625672423425],
 [0.020874480186016904,
  0.03633918693416842,
  0.035144418062164035,
  0.022795347193007645,
  -0.0036355519059021687],
 [0.0046785261641279895,
  0.035137990301026364,
  0.039626674214974296,
  0.0052167566297191235,
  -0.001790045130212442],
 [-0.017099748633392108,
  0.040769724249240476,
  0.03792197236263317,
  -0.02239763931707026,
  -0.008966750689521633],
 [-0.005628688171333402,
  0.013453899684012031,
  0.021284359536505988,
  -0.011853563500576677,
  0.0010526149079206992],
 [-0.008758808611354755,
  0.010047064406250474,
  0.011814573018157347,
  -0.009297043175992867,
  -0.012185106389492571],
 [-0.014217121757514991,
  0.017327239100780805,
  0.023207703479525693,
  -0.021183657444975773,
  -0.014396035724004145],
 [-0.0023428152912038